In [26]:
import pandas as pd
import urllib.request

data = pd.read_csv('../news/scraping_result/WSJ_result.csv', error_bad_lines=False)

In [27]:
data = data.append(pd.read_csv('../news/scraping_result/Forbes_result.csv', error_bad_lines=False))

In [28]:
data = data.append(pd.read_csv('../news/scraping_result/NYtimes_result.csv', error_bad_lines=False))

In [29]:
data = data.append(pd.read_csv('../news/scraping_result/Crypto News_result.csv', error_bad_lines=False))

In [30]:
data = data.append(pd.read_csv('../news/scraping_result/Coindesk_result.csv', error_bad_lines=False))

In [31]:
data = data.append(pd.read_csv('../news/scraping_result/Cointelegraph_result.csv', error_bad_lines=False))

In [32]:
len(data)

142

In [33]:
text = data[['main article']]

In [34]:
type(text)

pandas.core.frame.DataFrame

In [35]:
import nltk
text['main article'] = text.apply(lambda row: nltk.word_tokenize(row['main article']), axis=1)

/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['main article'] = text['main article'].apply(lambda x: [word for word in x if word not in (stop)])

/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
from nltk.stem import WordNetLemmatizer
text['main article'] = text['main article'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                        main article
0  [Not, long, ago, ,, bitcoin, universe, cryptoc...
1  [Almost, good, thing, invest, cryptocurrencies...
2  [A, Canadian, cryptocurrency, exchange, say, $...
3  [WASHINGTON—The, first, futures, contract, pay...
4  [At, begin, 2018, ,, question, whether, bitcoi...


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
tokenized_doc = text['main article'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

0    [long, bitcoin, universe, cryptocurrency, peer...
1    [Almost, good, thing, invest, cryptocurrencies...
2    [Canadian, cryptocurrency, exchange, million, ...
3    [WASHINGTON—The, first, futures, contract, cry...
4    [begin, 2018, question, whether, bitcoin, coul...
Name: main article, dtype: object


In [46]:
type(tokenized_doc)

pandas.core.series.Series

In [47]:
tokenized_doc = tokenized_doc.apply(lambda x : ' '.join(x))

In [48]:
tokenized_doc

0     long bitcoin universe cryptocurrency peer seem...
1     Almost good thing invest cryptocurrencies 2018...
2     Canadian cryptocurrency exchange million worth...
3     WASHINGTON—The first futures contract cryptocu...
4     begin 2018 question whether bitcoin could live...
                            ...                        
24    Signing Memorandum Understanding administrator...
25    Coin ECR20 token first announce release Septem...
26    Screenshot Bank England Twitter survey Source ...
27    appoint Unibright make preparations internal w...
28    Comcast currently work NBCUniversal test Block...
Name: main article, Length: 142, dtype: object

In [49]:
article_contents = tokenized_doc.values.tolist()

In [50]:
article_contents

['long bitcoin universe cryptocurrency peer seem light years away traditional market distant point assets trade laws physics capital investors traditional market flow cryptocurrencies make behave like traditional assets—just riskier',
 'Almost good thing invest cryptocurrencies 2018 break These assets crush year Bitcoin price example fall 15,000 4,000 U.S. code provide measure relief allow investors losses offset winners either',
 'Canadian cryptocurrency exchange million worth customers hold stick electronic vault company founder without share password independent researchers publicly available transaction record associate QuadrigaCX suggest money trap',
 'WASHINGTON—The first futures contract cryptocurrency rather cash expect soon regulatory approval contract launch York Stock Exchange owner Intercontinental Exchange Inc. institutional players stay cryptocurrency market concern unregulated susceptible manipulation Bitcoin',
 'begin 2018 question whether bitcoin could live hype 2017 m

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [52]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform([ " ".join(article_contents) ])

In [53]:
word2id = defaultdict(lambda: 0)

In [54]:
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

In [55]:
tf_idf_result = []
token_list = []

In [56]:
for i, sent in enumerate([" ".join(article_contents)]):
    for token in sent.split() :
        if token not in token_list :
            temp=[]
            temp.append(token)
            temp.append(sp_matrix[i, word2id[token]])
            token_list.append(token)
            tf_idf_result.append(temp)

In [57]:
from operator import itemgetter
for listitem in sorted(tf_idf_result,key=itemgetter(1),reverse=True) :
    print(listitem)

['bitcoin', 0.23456279185909917]
['blockchain', 0.19847620849616085]
['crypto', 0.15971654488411596]
['company', 0.15904827482183934]
['market', 0.15771173469728605]
['make', 0.15704346463500943]
['year', 0.13632709270443372]
['time', 0.1356588226421571]
['cryptocurrency', 0.12763958189483746]
['would', 0.11761553096068791]
['also', 0.11427418064930472]
['like', 0.11226937046247482]
['could', 0.10825975008881501]
['first', 0.10692320996426173]
['people', 0.10558666983970846]
['work', 0.09890396921694211]
['digital', 0.0948943488432823]
['technology', 0.09288953865645239]
['take', 0.09288953865645239]
['bank', 0.0915529985318991]
['money', 0.09088472846962248]
['trade', 0.08954818834506921]
['price', 0.08687510809596266]
['many', 0.08620683803368602]
['want', 0.08620683803368602]
['even', 0.0855385679714094]
['world', 0.08286548772230284]
['business', 0.07885586734864303]
['exchange', 0.0781875972863664]
['think', 0.07685105716181312]
['last', 0.07685105716181312]
['come', 0.07685105716

['elect', 0.0026730802491065435]
['alter', 0.0026730802491065435]
['frequently', 0.0026730802491065435]
['dummy', 0.0026730802491065435]
['2000', 0.0026730802491065435]
['commodity', 0.0026730802491065435]
['analyst', 0.0026730802491065435]
['cable', 0.0026730802491065435]
['hook', 0.0026730802491065435]
['publication', 0.0026730802491065435]
['rent', 0.0026730802491065435]
['victims', 0.0026730802491065435]
['apparently', 0.0026730802491065435]
['apartment', 0.0026730802491065435]
['ignore', 0.0026730802491065435]
['accountability', 0.0026730802491065435]
['optimism', 0.0026730802491065435]
['classic', 0.0026730802491065435]
['writers', 0.0026730802491065435]
['comedy', 0.0026730802491065435]
['switch', 0.0026730802491065435]
['endless', 0.0026730802491065435]
['reasonable', 0.0026730802491065435]
['principles', 0.0026730802491065435]
['island', 0.0026730802491065435]
['creator', 0.0026730802491065435]
['transition', 0.0026730802491065435]
['initiatives', 0.0026730802491065435]
['dapp

['NumberThis', 0.0013365401245532718]
['rising.Earlier', 0.0013365401245532718]
['BitGive', 0.0013365401245532718]
['GiveTrack', 0.0013365401245532718]
['non-profits.Whereas', 0.0013365401245532718]
['Tour', 0.0013365401245532718]
['one-of-a-kind', 0.0013365401245532718]
['Houston', 0.0013365401245532718]
['Area', 0.0013365401245532718]
['Women', 0.0013365401245532718]
['Center', 0.0013365401245532718]
['HAWC', 0.0013365401245532718]
['buzz—and', 0.0013365401245532718]
['batter', 0.0013365401245532718]
['momentum', 0.0013365401245532718]
['Established', 0.0013365401245532718]
['cutting-edge', 0.0013365401245532718]
['slice', 0.0013365401245532718]
['Malta', 0.0013365401245532718]
['first-ever', 0.0013365401245532718]
['DevCon', 0.0013365401245532718]
['event.Smaller', 0.0013365401245532718]
['Leading', 0.0013365401245532718]
['Toutiao', 0.0013365401245532718]
['Buzzfeed', 0.0013365401245532718]
['CherryHome', 0.0013365401245532718]
['Cherry', 0.0013365401245532718]
['2019.Nick', 0.0013

['Deals', 0.0013365401245532718]
['handouts.Moderating', 0.0013365401245532718]
['Radhika', 0.0013365401245532718]
['Iyengar-Emens', 0.0013365401245532718]
['StarChain', 0.0013365401245532718]
['Ventures.', 0.0013365401245532718]
['cases.', 0.0013365401245532718]
['investment.The', 0.0013365401245532718]
['Ritz', 0.0013365401245532718]
['Bits.', 0.0013365401245532718]
['QuarkChain', 0.0013365401245532718]
['Zhou', 0.0013365401245532718]
['Next', 0.0013365401245532718]
['Litecoins', 0.0013365401245532718]
['value.He', 0.0013365401245532718]
['summit', 0.0013365401245532718]
['specify', 0.0013365401245532718]
['quarter-million', 0.0013365401245532718]
['candy', 0.0013365401245532718]
['candy.An', 0.0013365401245532718]
['Bitrue', 0.0013365401245532718]
['half-dozen', 0.0013365401245532718]
['Bitrue.', 0.0013365401245532718]
['Curtis', 0.0013365401245532718]
['Wang', 0.0013365401245532718]
['percentage', 0.0013365401245532718]
['crowd.Someone', 0.0013365401245532718]
['TBILISI', 0.0013365

['Paris', 0.0013365401245532718]
['Snapshot', 0.0013365401245532718]
['Above', 0.0013365401245532718]
['Roman', 0.0013365401245532718]
['Catholic', 0.0013365401245532718]
['Panama', 0.0013365401245532718]
['five-day', 0.0013365401245532718]
['Pope', 0.0013365401245532718]
['Francis', 0.0013365401245532718]
['attend.In', 0.0013365401245532718]
['Russell', 0.0013365401245532718]
['two-time', 0.0013365401245532718]
['Pulitzer', 0.0013365401245532718]
['years.Oscar', 0.0013365401245532718]
['nominations', 0.0013365401245532718]
['Roma', 0.0013365401245532718]
['foreign-language', 0.0013365401245532718]
['Favourite', 0.0013365401245532718]
['surprises.Baseball', 0.0013365401245532718]
['Fame', 0.0013365401245532718]
['Mariano', 0.0013365401245532718]
['Rivera', 0.0013365401245532718]
['Yankees', 0.0013365401245532718]
['Series', 0.0013365401245532718]
['Fame.Serena', 0.0013365401245532718]
['victory', 0.0013365401245532718]
['seventh-seeded', 0.0013365401245532718]
['Karolina', 0.0013365401

['all…Key', 0.0013365401245532718]
['assumptions', 0.0013365401245532718]
['today.As', 0.0013365401245532718]
['others.The', 0.0013365401245532718]
['.Miners', 0.0013365401245532718]
['process.Yet', 0.0013365401245532718]
['progress.Let', 0.0013365401245532718]
['AntMiner', 0.0013365401245532718]
['Designed', 0.0013365401245532718]
['Classic', 0.0013365401245532718]
['.Monero', 0.0013365401245532718]
['true.That', 0.0013365401245532718]
['upgrades.The', 0.0013365401245532718]
['negotiations.Yet', 0.0013365401245532718]
['debates.Another', 0.0013365401245532718]
['Classic.In', 0.0013365401245532718]
['examples', 0.0013365401245532718]
['practice.But', 0.0013365401245532718]
['politics.While', 0.0013365401245532718]
['sector.From', 0.0013365401245532718]
['Administering', 0.0013365401245532718]
['reason.In', 0.0013365401245532718]
['system.What', 0.0013365401245532718]
['cryptocurrency.Separate', 0.0013365401245532718]
['them.Bitcoin', 0.0013365401245532718]
['corruption.From', 0.0013365

['fraudulently', 0.0006682700622766359]
['destruction', 0.0006682700622766359]
['environments', 0.0006682700622766359]
['inflexible', 0.0006682700622766359]
['unforeseen', 0.0006682700622766359]
['object', 0.0006682700622766359]
['supplementary', 0.0006682700622766359]
['scholar', 0.0006682700622766359]
['mistakenly', 0.0006682700622766359]
['intermediation', 0.0006682700622766359]
['unreliable', 0.0006682700622766359]
['trustworthy', 0.0006682700622766359]
['imbalances', 0.0006682700622766359]
['eliminate', 0.0006682700622766359]
['rigidity', 0.0006682700622766359]
['mechanize', 0.0006682700622766359]
['fond', 0.0006682700622766359]
['invoke', 0.0006682700622766359]
['paradox', 0.0006682700622766359]
['sociologist', 0.0006682700622766359]
['apparatuses', 0.0006682700622766359]
['authenticate', 0.0006682700622766359]
['cultivate', 0.0006682700622766359]
['wholesale', 0.0006682700622766359]
['slosh', 0.0006682700622766359]
['integral', 0.0006682700622766359]
['famously', 0.0006682700622

In [58]:
tf_idf_result

[['long', 0.049451984608471056],
 ['bitcoin', 0.23456279185909917],
 ['universe', 0.0006682700622766359],
 ['cryptocurrency', 0.12763958189483746],
 ['peer', 0.010692320996426174],
 ['seem', 0.030072152802448612],
 ['light', 0.005346160498213087],
 ['years', 0.05613468523123741],
 ['away', 0.01136059105870281],
 ['traditional', 0.022052912055128982],
 ['market', 0.15771173469728605],
 ['distant', 0.0013365401245532718],
 ['point', 0.04009620373659815],
 ['assets', 0.039427933674321514],
 ['trade', 0.08954818834506921],
 ['laws', 0.009355780871872901],
 ['physics', 0.0013365401245532718],
 ['capital', 0.03140869292700189],
 ['investors', 0.058139495418067316],
 ['flow', 0.007350970685042995],
 ['cryptocurrencies', 0.07150489666360003],
 ['make', 0.15704346463500943],
 ['behave', 0.004009620373659815],
 ['like', 0.11226937046247482],
 ['assets—just', 0.0013365401245532718],
 ['riskier', 0.0006682700622766359],
 ['Almost', 0.0013365401245532718],
 ['good', 0.03675485342521497],
 ['thing',

In [59]:
token_list

['long',
 'bitcoin',
 'universe',
 'cryptocurrency',
 'peer',
 'seem',
 'light',
 'years',
 'away',
 'traditional',
 'market',
 'distant',
 'point',
 'assets',
 'trade',
 'laws',
 'physics',
 'capital',
 'investors',
 'flow',
 'cryptocurrencies',
 'make',
 'behave',
 'like',
 'assets—just',
 'riskier',
 'Almost',
 'good',
 'thing',
 'invest',
 '2018',
 'break',
 'These',
 'crush',
 'year',
 'Bitcoin',
 'price',
 'example',
 'fall',
 '15,000',
 '4,000',
 'U.S.',
 'code',
 'provide',
 'measure',
 'relief',
 'allow',
 'losses',
 'offset',
 'winners',
 'either',
 'Canadian',
 'exchange',
 'million',
 'worth',
 'customers',
 'hold',
 'stick',
 'electronic',
 'vault',
 'company',
 'founder',
 'without',
 'share',
 'password',
 'independent',
 'researchers',
 'publicly',
 'available',
 'transaction',
 'record',
 'associate',
 'QuadrigaCX',
 'suggest',
 'money',
 'trap',
 'WASHINGTON—The',
 'first',
 'futures',
 'contract',
 'rather',
 'cash',
 'expect',
 'soon',
 'regulatory',
 'approval',
 '

In [60]:
from gensim import corpora
tokenized_doc = [d.split() for d in token_list]
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) 

[(1, 1)]


In [64]:
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.000*"happen" + 0.000*"forgiveness" + 0.000*"Crypto" + 0.000*"faster"')
(1, '0.000*"year.Bitcoin" + 0.000*"crypto-focused" + 0.000*"decades" + 0.000*"minute"')
(2, '0.000*"unaffiliated" + 0.000*"Here" + 0.000*"success" + 0.000*"previous"')
(3, '0.000*"stricter" + 0.000*"size" + 0.000*"revolutionize" + 0.000*"uncertainty"')
(4, '0.000*"billions" + 0.000*"specifically" + 0.000*"confuse" + 0.000*"mislead"')
(5, '0.000*"largest" + 0.000*"\'how" + 0.000*"create" + 0.000*"Minneapolis"')
(6, '0.000*"Republican" + 0.000*"something" + 0.000*"larger" + 0.000*"expansion"')
(7, '0.000*"almost" + 0.000*"settlements." + 0.000*"Wildeses" + 0.000*"receive"')
(8, '0.000*"storm" + 0.000*"speed" + 0.000*"move" + 0.000*"investment.Of"')
(9, '0.000*"Sheth" + 0.000*"geeks" + 0.000*"experience" + 0.000*"reporter"')


In [65]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
